In [1]:
from ipywidgets import interact,interactive,interact_manual
from matplotlib import pyplot as plt
from numpy.linalg import svd
import numpy as np

from skimage import data
from skimage.color import rgb2gray
from skimage import img_as_ubyte,img_as_float

from sklearn.decomposition import PCA, TruncatedSVD

%matplotlib inline

In [25]:
train = np.loadtxt(r'../data/MNIST_train_small.csv', delimiter=',')[:,1:]
test = np.loadtxt(r'../data/MNIST_test_small.csv', delimiter=',')[:150][:,1:]

In [59]:
def compress_image(img, k):
    """
     compresses gray scale images and display the reconstructed image.
     Also displays a plot of singular values
    """
    image = img
    original_shape = image.shape
    reconst_img,s = compress_svd(image,k)
    fig,axes = plt.subplots(1,2,figsize=(8,5))
    axes[0].plot(s)
    compression_ratio =100.0* (k*(original_shape[0] + original_shape[1])+k)/(original_shape[0]*original_shape[1])
    axes[1].set_title("compression ratio={:.2f}".format(compression_ratio)+"%")
    axes[1].imshow(reconst_img,cmap='gray')
    axes[1].axis('off')
    fig.tight_layout()

def compress_svd(image, k):
    """
    Perform svd decomposition and truncated (using k singular values/vectors) reconstruction
    returns
    --------
      reconstructed matrix reconst_matrix, array of singular values s
    """
    U,s,V = svd(image,full_matrices=False)
    reconst_matrix = np.dot(U[:,:k],np.dot(np.diag(s[:k]),V[:k,:]))
    return reconst_matrix, s

In [60]:
interact(compress_image, img=train[0].reshape(28, 28), k=(1,20));

TraitError: Invalid selection: value not found

In [50]:
for k in range(2, 11):
    imgs_trunc = []
    for img in train:
        img_r = img.reshape(28, 28)
        img_trunc, _ = compress_svd(img.reshape(28, 28), 2)
        imgs_trunc.append(img_trunc)
    np.savetxt(f'../data/train_{k}.csv', np.array(imgs_trunc).reshape((3,-1)), delimiter=',')

In [51]:
interact(compress_image, img=arr[0], k=(1,20));

interactive(children=(IntSlider(value=10, description='k', max=20, min=1), Output()), _dom_classes=('widget-in…